In [ ]:
# Standard library imports
import os
import random
import warnings

# Third-party imports
import cv2
import numpy as np
from keras import backend as K
from PIL import Image
from skimage.io import imread
import nbimporter
from unet_model import UNET
from skimage.transform import resize

# Setting up matplotlib to work interactively in a Jupyter environment
%matplotlib inline

# Setting the seed for reproducibility
seed_value = 42
random.seed(seed_value)  # For Python's built-in random module
np.random.seed(seed_value)  # For NumPy

# Don't show warning messages
warnings.filterwarnings('ignore')


In [ ]:
def pred_and_save_original_size(model, fn):
    # Read the image in grayscale
    image = cv2.imread(input_data_path + fn, 0)

    # Store the original image dimensions
    original_size = image.shape

    # Preprocess the image
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))/255.
    image = (image - mean)/std
    
    # Predict the mask
    test_preds = model.predict(image.reshape(1,IMG_WIDTH,IMG_HEIGHT,1), verbose = 0)
    preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)

    # Resize the mask to the original size of the image
    preds_test_thresh_resized = cv2.resize(preds_test_thresh[0], (original_size[1], original_size[0]))

    # Save the mask
#     cv2.imwrite(output_mask_path_orig_size + 'M' + fn, preds_test_thresh_resized*255)
    cv2.imwrite(output_mask_path_orig_size + fn, preds_test_thresh_resized*255)


#  Sample Usage

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:

      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    

In [ ]:
input_data_path = r'< PATH_TO_INPUT >'
output_mask_path_orig_size = r'< PATH_TO_INPUT_MASK >'

# Create empty arrays
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
IMG_CHANNELS = 1
per_image_norm = False

# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# load model
base_ptch_1024 = UNET(IMG_HEIGHT)
base_ptch_1024.model.load_weights('path_to_your_trained_model.h5')

# Get the list of test data
input_data_list = os.listdir(input_data_path)

# Perform prediction and save results
for fn in input_data_list:
    pred_and_save_original_size(base_ptch_1024.model, fn)